In [1]:
import json
import pandas as pd

In [2]:
def export_file(filename):
    print(f'Exporting {filename}')
    with open(f'SERIAL_CAPTURE/{filename}.json') as f:
        d = json.load(f)


    def strip_data(packet, len):
        hex = packet.split(":")
        hex = hex[-len:]
        return ''.join(hex)

    data = []

    for packet in d:

        data_len = int(packet['_source']['layers']['usb']['usb.data_len'])

        packet_str = packet['_source']['layers']['ftdi-ft']

        time = packet['_source']['layers']['frame']['frame.time_relative']

        elem = {}

        if "ftdi-ft.if_a_tx_payload" in packet_str:
            #tx
            elem = {
            "time": time,
            "dir": "tx",
            "packet": strip_data(packet_str["ftdi-ft.if_a_tx_payload"], data_len),
            "len": data_len,
            }
        else:
            #rx
            elem = {
            "time": time,
            "dir": "rx",
            "packet": strip_data(packet_str["ftdi-ft.if_a_rx_payload"], data_len),
            "len": data_len,
            }

        data.append(elem)

    df = pd.DataFrame(data)

    # Create an empy dataframe with the same columns as df
    df_concat = pd.DataFrame(columns=df.columns)

    #itterate over df 

    # COMINE PACKETS
    packet = ""
    length = len(df)
    for index, row in df.iterrows():
        direction = row['dir']
        packet = packet + row['packet']


        if index == length - 1:
            # Over 2 as two chars per byte
            df_row = pd.DataFrame([[row['time'], row['dir'], packet, int(len(packet)/2)]], columns=df.columns)
            df_concat = pd.concat([df_concat, df_row])
            packet = ""
            break
            

        if df.iloc[index+1]['dir'] != direction:
            df_row = pd.DataFrame([[row['time'], row['dir'], packet, int(len(packet)/2)]], columns=df.columns)
            df_concat = pd.concat([df_concat, df_row])
            packet = ""
        

    df_concat.to_csv(f'SERIAL_CAPTURE/{filename}.csv', index=False)

In [3]:
#get list of file in directory with extension json and remove extension

# WIRESHARK FILTER -> ftdi-ft&&(ftdi-ft.if_a_rx_payload||ftdi-ft.if_a_tx_payload)
import os
files = [f.split('.')[0] for f in os.listdir('SERIAL_CAPTURE') if f.endswith('.json')]

for file in files:
    export_file(file)

Exporting capp_press_prog_nobean
Exporting button_press_mocha
Exporting button_press_other
Exporting button_press_white_coffee
Exporting read_errors
Exporting button_press_cappuccino
Exporting button_press_hot_chocolate
Exporting americano_press_prog_goodrun_then_nobeans
Exporting capp_press_prog_goodrun
Exporting americano_press
Exporting button_press_latte_works
Exporting button_press_hot_water
Exporting button_press_latte


In [15]:
tx = "f0f20503000100000001008145000003000000130009000000090191640114000103643700010000005500c8001e00320000080000000f10055f0000000303000f016300010100004c617474652020202020202020202020202020003200000005001701010101640114000104643800010000004600be002300320000080014000f10055f0000050303000f0134000102000043617070756363696e6f20202020202020202000320000000500d80009010164011400010364370001000000e60069000000320100080000000f32055f0000000303000f01410001000000576869746520436f666665652020202020202000320000000500340003010164010000010064390001000000180100000000320100080000000f32055f0000000000000f01460001000000416d65726963616e6f2020202020202020202000320000000500c6001b010164011400010964340001000000230064000000320000080014000f10055f3400050300010f014100010200004d6f6368612020202020202020202020202020003200aa0005002a01200501640100000009641e00000001000a0069000000000100080000000f1000012d00000000010a010a0001000000486f742043686f636f6c617465202020202020003200be0000000801120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000130201640100000000642100000000000a00b4000000000000080000000f1000010000000000020a01140001000000686f7420776174657220202020202020202020003200000005003110120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000020101640114000104644c0001000000420096001e00320000080014000f32055f0000050303000f012b0001020000322043617070756363696e6f20202020202020003200000005000000080101640114000100645a0001000000460000000000320000080014000f32055f0000050000000f012300010200003220657370726573736f2020202020202020200032000000050000000a0101650114000103643d00010000003c00ae002d00320000080000000f10055f0000000303000f01330001010100322063616665206c6174746520202020202020003200000005000000040101640100000100644b00010000000e0100000000320000080000000f32055f0000000000000f013d0001010000322063616665206372656d65202020202020200032000000050000000f0301640100000001641e00000000000a005a000000000000080000000f1000010000000000020a011400010000006d696c6b206672656520666c6f7720202020200032000000050000001b010164011400010964370001000000200050000000320000080014000f32055f3e00050300030f012f000102000063686f63696174746f20202020202020202020003200500005000000120001640100000000641e00010000000a0000000000000000080000000f1000010000000000000a01140001000000626c6f636b6564202020202020202020202020003200000005000000130201640100000000642100010000000a0078003c00000000000000000f0f00010000000000030a01140001000000486f7420776174657220202020202020202020003200000005000000000a11000000000000001e00000000000a0000000000000000000000000f0f000000000000000000001400000000007832202020202020202020202020202020202000000000000a000000000a11000000000000001e00000000000a0000000000000000000000000f0f000000000000000000001400000000007832202020202020202020202020202020202000000000000a000000db"


n= 2
chunks = [tx[i:i+n] for i in range(0, len(tx), n)]

#print(chunks)
dec = []
for byte in chunks:
    dec.append(int(byte, 16))

#print(dec)

a = "240,97,0,3,0,1,0,0,0,1,0,129,69,0,0,1,0,3,0,0,0,9,0,0,0,3,1,17,100,1,0,0,1,0,100,57,0,1,0,0,0,24,1,0,0,0,0,50,1,0,8,0,0,0,15,50,5,95,0,0,0,0,0,0,15,1,70,0,1,0,0,0,65,109,101,114,105,99,97,110,111,32,32,32,32,32,32,32,32,32,32,0,50,0,0,0,5,0,212,0,27"
b = "240,97,0,3,0,1,0,0,0,1,0,129,69,0,0,1,0,3,0,0,0,9,0,0,0,3,1,1,100,1,0,0,1,0,100,57,0,1,0,0,0,24,1,0,0,0,0,50,1,0,8,0,0,0,15,50,5,95,0,0,0,0,0,0,15,1,70,0,1,0,0,0,65,109,101,114,105,99,97,110,111,32,32,32,32,32,32,32,32,32,32,0,50,0,0,0,5,0,212,0,11"
c = "240,97,0,3,0,1,0,0,0,1,0,129,69,0,0,1,0,3,0,0,0,9,0,0,0,3,1,17,100,1,0,0,1,0,100,57,0,1,0,0,0,24,1,0,0,0,0,50,1,0,8,0,0,0,15,50,5,95,0,0,0,0,0,0,15,1,70,0,1,0,0,0,65,109,101,114,105,99,97,110,111,32,32,32,32,32,32,32,32,32,32,0,50,0,0,0,5,0,212,0,27"
d = "240,97,0,3,0,1,0,0,0,1,0,129,69,0,0,1,0,3,0,0,0,9,0,0,0,3,1,17,100,1,0,0,1,0,100,57,0,1,0,0,0,24,1,0,0,0,0,50,1,0,8,0,0,0,15,50,5,95,0,0,0,0,0,0,15,1,70,0,1,0,0,0,65,109,101,114,105,99,97,110,111,32,32,32,32,32,32,32,32,32,32,0,50,0,0,0,5,0,212,0,27"
e = "240,97,0,3,0,1,0,0,0,1,0,129,69,0,0,1,0,3,0,0,0,9,0,0,0,3,1,1,100,1,0,0,1,0,100,57,0,1,0,0,0,24,1,0,0,0,0,50,1,0,8,0,0,0,15,50,5,95,0,0,0,0,0,0,15,1,70,0,1,0,0,0,65,109,101,114,105,99,97,110,111,32,32,32,32,32,32,32,32,32,32,0,50,0,0,0,5,0,212,0,11"
f = "240,97,0,3,0,1,0,0,0,1,0,129,69,0,0,1,0,3,0,0,0,9,0,0,0,3,1,145,100,1,0,0,1,0,100,57,0,1,0,0,0,24,1,0,0,0,0,50,1,0,8,0,0,0,15,50,5,95,0,0,0,0,0,0,15,1,70,0,1,0,0,0,65,109,101,114,105,99,97,110,111,32,32,32,32,32,32,32,32,32,32,0,50,0,0,0,5,0,213,0,156"
g = "240,97,0,3,0,1,0,0,0,1,0,129,69,0,0,1,0,3,0,0,0,9,0,0,0,3,1,17,100,1,0,0,1,0,100,57,0,1,0,0,0,24,1,0,0,0,0,50,1,0,8,0,0,0,15,50,5,95,0,0,0,0,0,0,15,1,70,0,1,0,0,0,65,109,101,114,105,99,97,110,111,32,32,32,32,32,32,32,32,32,32,0,50,0,0,0,5,0,213,0,28"

print(b == e)

True


In [16]:
import struct

# byte to value conversions

def convert_b_signed(byte):
    return struct.unpack('b', bytes.fromhex(byte))[0]

def convert_b_unsigned(byte):
    return struct.unpack('B', bytes.fromhex(byte))[0]

def convert_b_char(byte):
    return struct.unpack('c', bytes.fromhex(byte))[0]


def convert_unsigned(hex):
    dec = int(hex, 16)
    return dec

def convert_signed(hex):
    bits = len(hex) * 4
    dec = int(hex, 16)
    if dec > (bits-1)**2:
        dec = dec - bits**2
    return dec

def convert_2b_float(byte_2):
    return struct.unpack('e', bytes.fromhex(byte_2))[0]

def convert_4b_float(byte_4):
    return struct.unpack('f', bytes.fromhex(byte_4))[0]

def convert_8b_double(byte_8):
    return struct.unpack('d', bytes.fromhex(byte_8))[0]

def convert_string(byte):
    return struct.unpack('s', bytes.fromhex(byte))[0]


def convert_byte(byte):
    char = convert_b_char(byte)
    ub = convert_unsigned(byte)
    sb = convert_signed(byte)

    return [sb , ub, char]

def convert_2b(byte_2):
    u2 = convert_unsigned(byte_2)
    s2 = convert_signed(byte_2)
    float_s = convert_2b_float(byte_2)
    return [s2, u2, float_s ]

def convert_4b(byte_4):
    u4 = convert_unsigned(byte_4)
    s4 = convert_signed(byte_4)
    f = convert_4b_float(byte_4)

    return [ s4, u4, f]

def convert_8b(byte_8):
    u8 = convert_unsigned(byte_8)
    s8 = convert_signed(byte_8)
    double = convert_8b_double(byte_8)

    return [s8, u8, double]


def window_hexstring(data):
    b =  [data[i:i+2] for i in range(0, len(data) - 1)]
    b2 = [data[i:i+4] for i in range(0, len(data) - 3)]
    b4 = [data[i:i+8] for i in range(0, len(data) - 7)]
    b8 =  [data[i:i+16] for i in range(0, len(data) - 15)]

    b_conversion = pd.DataFrame(columns=['byte', 'signed', 'unsigned', 'char'])
    b2_conversion = pd.DataFrame(columns=['2byte', 'signed_h', 'unsigned_h',  'float_s'])
    b4_conversion = pd.DataFrame(columns=['4byte', 'signed', 'unsigned' , 'float'])
    b8_conversion = pd.DataFrame(columns=['8byte', 'signed_l', 'unsigned_l', 'double'])

    string = ""# convert_string(data)

    for i, byte in enumerate(b):
        b_conversion.loc[i] = [byte] + convert_byte(byte)

    for i, byte_2 in enumerate(b2):
        b2_conversion.loc[i] = [byte_2] + convert_2b(byte_2)
    
    for i, byte_4 in enumerate(b4):
        b4_conversion.loc[i] = [byte_4] + convert_4b(byte_4)
    
    for i, byte_8 in enumerate(b8):
        b8_conversion.loc[i] = [byte_8] + convert_8b(byte_8)

    return b_conversion, b2_conversion, b4_conversion, b8_conversion, string


cap = "f01700050001000000010005490100000000000000000000000157"
ame = "f016000500010000000100054902000000000000000000000057"

b, b2, b4, b8, s = window_hexstring(cap)

print(b)
print(b2)


   byte  signed  unsigned     char
0    f0     176       240  b'\xf0'
1    01       1         1  b'\x01'
2    17      23        23  b'\x17'
3    70      48       112     b'p'
4    00       0         0  b'\x00'
5    00       0         0  b'\x00'
6    05       5         5  b'\x05'
7    50      16        80     b'P'
8    00       0         0  b'\x00'
9    00       0         0  b'\x00'
10   01       1         1  b'\x01'
11   10      16        16  b'\x10'
12   00       0         0  b'\x00'
13   00       0         0  b'\x00'
14   00       0         0  b'\x00'
15   00       0         0  b'\x00'
16   00       0         0  b'\x00'
17   00       0         0  b'\x00'
18   01       1         1  b'\x01'
19   10      16        16  b'\x10'
20   00       0         0  b'\x00'
21   00       0         0  b'\x00'
22   05       5         5  b'\x05'
23   54      20        84     b'T'
24   49       9        73     b'I'
25   90      80       144  b'\x90'
26   01       1         1  b'\x01'
27   10      16     